## Questionnaires Data Analysis

#### Created on 10/28/2023 <br/> Created by Daon Lee (onlyforres@gmail.com)

In [1]:
### preparation

import os
import glob
import numpy as np
import pandas as pd


### set path: bhv or physio 여부 변경 필!

datapath = r'/Users/daonly/Library/Mobile Documents/com~apple~CloudDocs/Res/Conformity/data/bhv'
savepath = r'/Users/daonly/Library/Mobile Documents/com~apple~CloudDocs/Res/Conformity/analysis/bhv'

#datapath = r'C:\Users\LSD\Desktop\Res\Conformity\data\bhv'
#savepath = r'C:\Users\LSD\Desktop\Res\Conformity\analysis\bhv'

print(np.__version__)

1.24.1


### Generate group file

In [24]:
### basic setting

taskname = 'Questionnaire'
quesdf = pd.DataFrame()

### raw file setting: range 설정 변경 필!

rawdf = pd.read_excel(datapath+'/quesData_pilotII.xlsx')
    
    ## SubID
quesdf.insert(0, "SubID", rawdf["SubID"])

    ## Physio
quesdf['drink_day'] = rawdf['drink_day']
quesdf['drink_amount'] = rawdf['drink_amount']
quesdf['workout_heavy'] = rawdf['workout_heavy']
quesdf['workout_normal'] = rawdf['workout_normal']
quesdf['workout_light'] = rawdf['workout_light']
quesdf['caffeine'] = rawdf['caffeine']
quesdf['workout_before'] = rawdf['workout_before']
quesdf['sleep_dura'] = rawdf['sleep_dura']
quesdf['wakeup'] = rawdf['sleep_time'].astype(float)+rawdf['sleep_dura'].astype(float)
quesdf['height'] = rawdf['height']
quesdf['weight'] = rawdf['weight']

    ## SVO
cols = [col for col in rawdf.columns if col.startswith('SVO')]
s_mean = rawdf[cols].apply(lambda x: np.mean([int(val.split('/')[0]) for val in x]), axis=1)
o_mean = rawdf[cols].apply(lambda x: np.mean([int(val.split('/')[1]) for val in x]), axis=1)

quesdf['SVOnum'] = np.degrees(np.arctan2(o_mean-50, s_mean-50))
conditions = [quesdf['SVOnum'] > 57.15, quesdf['SVOnum'] > 22.45, quesdf['SVOnum'] > -12.04]
choices = ['A', 'P', 'I']
quesdf['SVO'] = np.select(conditions, choices, default='C')

    ## BFNE-2
value_mapping = {'전혀 그렇지 않다':0, '그렇지 않다':1, '보통이다':2, '그렇다':3, '매우 그렇다':4}
change_columns = [col for col in rawdf.columns if col.startswith('BFNE')]
rawdf[change_columns] = rawdf[change_columns].replace(value_mapping)

quesdf['BFNE'] = rawdf.loc[:, 'BFNE1':'BFNE11'].sum(axis=1)

    ## K-FPES
change_columns = [col for col in rawdf.columns if col.startswith('FPES')]
rawdf[change_columns] = rawdf[change_columns]-1

quesdf['FPES'] = rawdf.loc[:, 'FPES1':'FPES8'].drop('5. 다른 사람이 흥미로워하리라 생각되는 이야깃거리가 있을 경우, 그것을 이야기하는 편이다.', axis=1).sum(axis=1)

    ## RSES
value_mapping = {'거의 그렇지 않다':1, '대체로 그렇지 않다':2, '대체로 그렇다':3, '정말 그렇다':4}
change_columns = [col for col in rawdf.columns if col.startswith('RSESP')]
rawdf[change_columns] = rawdf[change_columns].replace(value_mapping)
value_mapping = {'거의 그렇지 않다':4, '대체로 그렇지 않다':3, '대체로 그렇다':2, '정말 그렇다':1}
change_columns = [col for col in rawdf.columns if col.startswith('RSESN')]
rawdf[change_columns] = rawdf[change_columns].replace(value_mapping)

columns = [col for col in rawdf.columns if col.startswith('RSESP')]
quesdf['RSES_pos'] = rawdf[columns].sum(axis=1)
columns = [col for col in rawdf.columns if col.startswith('RSESN')]
quesdf['RSES_neg'] = rawdf[columns].sum(axis=1)
quesdf['RSES_sum'] = quesdf['RSES_pos'] + quesdf['RSES_neg']

    ## TEQ
value_mapping = {4:0, 3:1, 2:2, 1:3, 0:4}
change_columns = ['TEQ2', 'TEQ4', 'TEQ7', 'TEQ10', 'TEQ11', 'TEQ12', 'TEQ14', 'TEQ15']
rawdf[change_columns] = rawdf[change_columns].replace(value_mapping)

columns = [col for col in rawdf.columns if col.startswith('TEQ')]
quesdf['TEQ'] = rawdf[columns].sum(axis=1)

    ## K-MAIA
quesdf['MAIA_Sum'] = rawdf.loc[:, 'MAIA1':'MAIA32'].sum(axis=1)
quesdf['MAIA_Not'] = rawdf.loc[:, 'MAIA1':'MAIA6'].sum(axis=1)
quesdf['MAIA_Acc'] = rawdf.loc[:, 'MAIA7':'MAIA10'].sum(axis=1)
quesdf['MAIA_Att'] = rawdf.loc[:, 'MAIA11':'MAIA17'].sum(axis=1)
quesdf['MAIA_Min'] = rawdf.loc[:, 'MAIA18':'MAIA22'].sum(axis=1)
quesdf['MAIA_Ret'] = rawdf.loc[:, 'MAIA23':'MAIA29'].sum(axis=1)
quesdf['MAIA_Tru'] = rawdf.loc[:, 'MAIA30':'MAIA32'].sum(axis=1)

    ## MFQ
change_columns = [col for col in rawdf.columns if col.startswith('MFQ')]
rawdf[change_columns] = rawdf[change_columns]-1

quesdf['MFQ_Harm'] = rawdf[['MFQ1', 'MFQ7', 'MFQ12', 'MFQ17', 'MFQ23', 'MFQ28']].sum(axis=1)
quesdf['MFQ_Fairness'] = rawdf[['MFQ2', 'MFQ8', 'MFQ13', 'MFQ18', 'MFQ24', 'MFQ29']].sum(axis=1)
quesdf['MFQ_Ingroup'] = rawdf[['MFQ3', 'MFQ9', 'MFQ14', 'MFQ19', 'MFQ25', 'MFQ30']].sum(axis=1)
quesdf['MFQ_Author'] = rawdf[['MFQ4', 'MFQ10', 'MFQ15', 'MFQ20', 'MFQ26', 'MFQ31']].sum(axis=1)
quesdf['MFQ_Purity'] = rawdf[['MFQ5', 'MFQ11', 'MFQ16', 'MFQ21', 'MFQ27', 'MFQ32']].sum(axis=1)
quesdf['MFQ_Ctrl'] = rawdf[['MFQ6', 'MFQ22']].sum(axis=1)

    ## MBTI
quesdf['MBTI1'] = rawdf['MBTI'].str[0]
quesdf['MBTI2'] = rawdf['MBTI'].str[1]
quesdf['MBTI3'] = rawdf['MBTI'].str[2]
quesdf['MBTI4'] = rawdf['MBTI'].str[3]

    ## Handedness
quesdf['Hand'] = rawdf['Hand']

In [25]:
quesdf

,SubID,drink_day,drink_amount,workout_heavy,workout_normal,workout_light,caffeine,workout_before,sleep_dura,wakeup,...,MFQ_Fairness,MFQ_Ingroup,MFQ_Author,MFQ_Purity,MFQ_Ctrl,MBTI1,MBTI2,MBTI3,MBTI4,Hand
0,11,1,12,0,1,6,아니오,아니오,7,19.5,...,22,16,14,13,5,E,N,F,J,R
1,12,0,7,2,2,3,예,예,6,8.0,...,19,14,18,18,6,E,S,T,J,R
2,13,0,2,1,1,6,예,아니오,8,18.0,...,20,15,14,14,5,I,N,T,J,L
3,14,0,2,0,2,5,아니오,아니오,8,10.0,...,26,28,21,22,5,I,N,F,P,R
4,15,1,7,2,2,4,아니오,아니오,7,9.0,...,25,15,18,21,6,I,N,F,J,R
5,16,1,5,0,1,7,아니오,아니오,8,9.0,...,17,18,20,14,5,E,S,T,P,R
6,17,1,15,1,2,4,아니오,아니오,7,10.0,...,15,14,14,8,5,E,S,T,J,R
7,18,1,6,0,0,2,아니오,아니오,9,11.0,...,18,18,16,10,6,E,S,T,P,R
8,19,0,3,0,0,5,아니오,아니오,9,32.0,...,6,5,9,13,5,I,N,F,P,R
9,20,1,14,2,4,7,아니오,아니오,10,14.0,...,25,15,16,14,5,E,S,T,P,R


In [26]:
### save the quesdf as an excel file

savefile = f'{datapath}/adjQuesData_II.xlsx'
quesdf.to_excel(savefile, index=False, header=True)
print('Complete!')

Complete!
